Теперь с помощью LLM создадим датасет из запросов и ответов, чтобы по нему уже оценивать метрики системы поиска.

In [1]:
import numpy as np
import pandas as pd
import json
import regex
import torch
from torch import Tensor
from sentence_transformers import CrossEncoder
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
torch.manual_seed(142)
path = 'articles_links.json'
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(device)

cpu


In [3]:
model_name = 'HuggingFaceTB/SmolLM3-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
title = 'Новый гидрогель позволит хранить лекарства вне холодильника на протяжении нескольких недель'
with open('prompt.txt', 'r', encoding='UTF-8') as f:
    prompt = f.read()

prompt = prompt + f'{title}\nЗапросы:\n'
print(prompt)

Ты помощник для создания поисковых запросов. На основе заголовка новости сгенерируй ровно 2 возможных поисковых запроса, которые могли бы ввести пользователи.
Заголовок: Кружащий рядом с Землей астероид оказался выбитым куском Луны
Запросы:
1. происхождение астероидов
2. новости про околоземные космические объекты
Заголовок: Виртуальный велосипед помог реальным пациентам в реабилитации после инсульта
Запросы:
1. достижения в методах реабилитации после инсульта
2. технологии виртуальной реальности в медицине
Заголовок: В глобальном потеплении увидели неожиданные плюсы для зеленых черепах
Запросы:
1. статьи о глобальном потеплении
2. состояние популяции зеленых черепах
Заголовок: Физики открыли новый перспективный магнитный материал
Запросы:
1. современные перспективные достижения физики
2. новые магнитные материалы
Заголовок: Химики научились управлять люминесценцией редкоземельных металлов
Запросы:
1. заметки про исследования люминесценции
2. химия редкоземельных металлов
Заголовок: Ар

In [33]:
with open('prompt.txt', 'r', encoding='UTF-8') as f:
    prompt = f.read()

def clean_query(x):
    return [q[4:].strip() for q in x]

def query_generator(titles, prompt, gen_model=gen_model, gen_config=gen_config, tokenizer=tokenizer, device=device):
    prompts = [prompt + f'{title}\nЗапросы:\n' for title in titles]
    inputs = tokenizer(
        prompts,
        return_tensors='pt',
        padding=True,
        truncation=True
    ).to(device)
    outputs = gen_model.generate(**inputs, **gen_config)
    gen_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    results = [text.split('Запросы:')[-1] for text in gen_texts]
    results = [regex.findall(r'\n[12]\..*', text) for text in results]

    queries = [clean_query(text) for text in results]
    
    return queries

In [34]:
gen_config = {
    'max_new_tokens': 30,
    'num_beams': 4,
    'early_stopping': True,
    'do_sample': False
}

x = query_generator([title], prompt, gen_config=gen_config)
print(x)

[['достижения в области гидрогелей', 'хранение лекарств без холодильника']]


In [6]:
all_data = pd.read_json(path)
df = all_data[['title', 'link', 'annot']]
df

,title,link,annot
0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...
1,"Бактерии создали из себя батарейку, чтобы разр...",https://naked-science.ru/article/biology/bacte...,Они использовали содержащееся в почве железо. ...
2,Бактерии и добавки пероксида кальция сделают д...,https://naked-science.ru/article/column/bakter...,Ученые РГУ нефти и газа (НИУ) имени И.М. Губки...
3,Российские физики предложили искать темную мат...,https://naked-science.ru/article/column/temnuy...,Большой коллектив российских ученых из ведущих...
4,Белорусские ученые вырастили кожу из грибов и ...,https://naked-science.ru/article/column/beloru...,Специалисты UST Inc. разработали технологию по...
...,...,...,...
8267,В Испании нашли свой Стоунхендж,https://naked-science.ru/article/anthropology/...,"На поле, предназначенном для выращивания авока..."
8268,Астрономы оценили максимально возможное число ...,https://naked-science.ru/article/astronomy/mak...,"Ученые подсчитали, что без больших изменений о..."
8269,Рецепт сознания: могут ли люди и роботы мечтат...,https://naked-science.ru/article/hi-tech/retse...,В XXI веке вопрос о природе сознания станет од...
8270,У берегов Западной Африки нашли древний ударны...,https://naked-science.ru/article/geology/udarn...,"Ученые допускают, что 66 миллионов лет назад с..."


In [7]:
len(df['title']), len(set(df['title']))

(8272, 8272)

In [35]:
some_titles = list(df['title'].sample(4))
some_titles

['Ученые научились предсказывать 3D-траекторию опасных для землян выбросов плазмы',
 'Мыши-самцы использовали самок, чтобы спастись от нападения',
 'Нейробиологи нашли в мозгу выключатель апатии',
 'Муравьи могут строить ориентиры, обозначая дорогу домой']

In [36]:
some_queries = query_generator(some_titles, prompt, gen_config=gen_config)

In [37]:
print(*some_queries, sep='\n')

['прогнозирование выбросов плазмы', 'научные достижения в области космической плазмы']
['статьи о поведении мышей-самцов', 'примеры выживания в условиях опасности']
['новости в области нейробиологии', 'выключатель апатии в мозгу']
['исследования поведения муравьев', 'навигационные способности муравьев']


In [38]:
import json

some_results = [{'request': req, 'titles': head} for req, head in zip(some_queries, some_titles)]
with open('some_results.json', 'w', encoding='UTF-8') as f:
            json.dump(some_results, f, ensure_ascii=False, indent=4)

In [4]:
def clean_text(text, pattern=r'[^\w\p{P}]+'):
  pattern = regex.compile(pattern)
  text = pattern.sub(' ', text)

  return regex.sub(r'\s+', ' ', text).strip()

In [5]:
texts = [clean_text(t) for t in df['annot']]
passages = [f'passage: {t}' for t in texts]
texts[:5]

['Голые землекопы живут почти 40 лет — невероятный срок для грызуна размером с мышь. Недавнее открытие китайских ученых показало, что долголетие этим животным обеспечивает необычная версия одного белка, который не просто защищает организм, а буквально чинит его изнутри — ускоряет восстановление ДНК. Новые данные могут помочь продлить и человеческую жизнь.',
 'Они использовали содержащееся в почве железо. Днем бактерии захватывали фотоны, ночью — боролись с мешающими им жить соединениями.',
 'Ученые РГУ нефти и газа (НИУ) имени И.М. Губкина разработали новый метод обезвреживания нефтесодержащих отходов в Арктической зоне России. Он основан на комбинированном использовании бактериальных препаратов с добавками пероксида кальция, источника кислорода при обезвреживании отходов с использованием геотубных технологий. Технология успешно прошла полевые испытания.',
 'Большой коллектив российских ученых из ведущих физических институтов Москвы, Нижнего Новгорода, Сарова и Санкт-Петербурга предста